# Calculate global-mean diagnostics of LFRic-Atmosphere THAI runs

## Import libraries

External libraries

In [1]:
import pandas as pd

In [2]:
from aeolus.calc import bond_albedo, spatial_mean
from aeolus.core import AtmoSim
from aeolus.io import load_data
from aeolus.model import lfric, um

Local modules

In [3]:
import paths
from shared import MODELS, THAI_CASES

## Load data

In [4]:
# Skip UM (use numbers from the THAI papers instead)
_ = MODELS.pop("um")

Load LFRic-Atmosphere data for all 4 THAI cases and store it in a dictionary

In [5]:
runs = {}
for model_key, model_prop in MODELS.items():
    model = model_prop["model"]
    runs[model_key] = {}

    for sim_label, sim_prop in THAI_CASES.items():
        dset = load_data(
            paths.data_final
            / model_key
            / sim_label
            / f"{sim_label}_{sim_prop['proc_fname_suffix']}_time_mean.nc"
        )

        runs[model_key][sim_label] = AtmoSim(
            dset,
            name=sim_label,
            vert_coord="z",
            planet=sim_prop["planet"],
            const_dir=paths.const,
            model=model,
        )

## Apply diagnostics

Define diagnostics and their metadata

In [6]:
DIAGS = {
    "t_sfc": {
        "recipe": lambda AS: spatial_mean(AS.t_sfc, model=lfric),
        "fmt": lambda x: f"{round(x):.0f}",
        "title": r"T\textsubscript{s}",
    },
    "toa_olr": {
        "recipe": lambda AS: spatial_mean(AS.toa_olr, model=lfric),
        "fmt": lambda x: f"{round(x):.0f}",
        "title": r"TOA OLR",
    },
    "alb": {
        "recipe": lambda AS: spatial_mean(bond_albedo(AS._cubes, model=lfric)),
        "fmt": lambda x: f"{x:.2f}",
        "title": r"$\alpha_\text{p}$",
    },
}

Loop over cases, models, and diagnostics to populate a pandas `DataFrame`.

In [7]:
results = {}
for sim_label, sim_prop in THAI_CASES.items():
    dfs = {}
    for model_key, model_prop in MODELS.items():
        model = model_prop["model"]
        _data = {}
        for diag_key, diag_prop in DIAGS.items():
            try:
                _data[diag_prop["title"]] = float(
                    diag_prop["recipe"](runs[model_key][sim_label]).data
                )
            except iris.exceptions.ConstraintMismatchError:
                _data[diag_prop["title"]] = pd.NA
        dfs[model_prop["title"]] = pd.DataFrame(
            _data, index=pd.Index(name="GCM", data=[model_prop["title"]])
        )
    results[sim_label] = pd.concat(dfs, axis="index").droplevel(0).T

Print out the `DataFrame` as a LaTeX table

In [8]:
for sim_label, sim_prop in THAI_CASES.items():
    df = results[sim_label].T
    formatters = {
        k: {j["title"]: j["fmt"] for j in DIAGS.values()}[k] for k in df.columns
    }
    column_format = "l" + "c" * len(DIAGS)
    print(
        rf"{{}} & \multicolumn{{{len(DIAGS)}}}{{c}}{{{sim_prop['title']}}} \\", end="\r"
    )
    latex_str = df.to_latex(
        formatters=formatters,
        column_format=column_format,
        escape=False,
    )
    latex_str = latex_str.replace(rf"\begin{{tabular}}{{{column_format}}}", "")
    latex_str = latex_str.replace("\end{tabular}\n", "")
    latex_str = latex_str.replace("toprule", "tophline")
    latex_str = latex_str.replace("midrule", "middlehline")
    latex_str = latex_str.replace("bottomrule", "bottomhline")
    print(latex_str)

{} & \multicolumn{3}{c}{Ben 1} \\
\tophline
 & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ \\
GCM &  &  &  \\
\middlehline
LFRic-Atmosphere & 207 & 161 & 0.29 \\
\bottomhline

{} & \multicolumn{3}{c}{Ben 2} \\
\tophline
 & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ \\
GCM &  &  &  \\
\middlehline
LFRic-Atmosphere & 238 & 182 & 0.19 \\
\bottomhline

{} & \multicolumn{3}{c}{Hab 1} \\
\tophline
 & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ \\
GCM &  &  &  \\
\middlehline
LFRic-Atmosphere & 242 & 187 & 0.17 \\
\bottomhline

{} & \multicolumn{3}{c}{Hab 2} \\
\tophline
 & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ \\
GCM &  &  &  \\
\middlehline
LFRic-Atmosphere & 278 & 206 & 0.10 \\
\bottomhline

